In [1]:
import sys
import os
from os.path import join
import torch
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from ipywidgets import interact_manual,interact, fixed
import math


# Preliminary Uncertainty Tests

Functions

In [2]:
def extract_tensors(path, tensor_name):
    '''extracts tensors from path (as created when any uncertainty test is ran'''
    # get only the correct folders
    folders = []
    for sub_path in os.listdir(path):
        if sub_path.startswith('image'):
            folders.append( sub_path)
    
    # extract tensors
    all_images = {}
    for im_folder in folders:
        im_path = join(path, im_folder)
        # save the tensors in their own dict according to name before
        for tensor in os.listdir(im_path):
            if tensor == tensor_name:
                tensor_path = join(im_path, tensor)
                all_images[int(im_folder.split('_')[-1])] = torch.load(tensor_path, map_location = 'cpu')
    return all_images


In [3]:
def display_tensor(tensor_dict:dict, im_dict:dict, target_dict:dict, im_num: int = 0 , vmin:float = 0, vmax: float = 1):
    fig, ax = plt.subplots(1,3, figsize = (18, 12))
    im1 = ax[0].imshow(im_dict[im_num], cmap = 'gray',)
    im2 = ax[1].imshow(tensor_dict[im_num][0][0].numpy(), cmap = 'gray', vmin = vmin, vmax = vmax)
    im3 = ax[2].imshow(target_dict[im_num], cmap = 'gray', )
    ax[0].set_title('Orig')
    ax[1].set_title('Expected')
    ax[2].set_title('GT')
    fig.show()
    print('Min: ' ,torch.min(tensor_dict[im_num][0][0]).item())
    print('Max: ' ,torch.max(tensor_dict[im_num][0][0]).item())

In [4]:
def compare_tensor(tensor_dict1:dict, tensor_dict2:dict, im_dict:dict, target_dict:dict, im_num: int = 0 , vmin:float = 0, vmax: float = 1):
    fig, ax = plt.subplots(1,4, figsize = (30, 28))
    im1 = ax[0].imshow(im_dict[im_num], cmap = 'gray',)
    im2 = ax[1].imshow(tensor_dict1[im_num][0][0].numpy(), cmap = 'gray', vmin = vmin, vmax = vmax)
    im3 = ax[2].imshow(tensor_dict2[im_num][0][0].numpy(), cmap = 'gray', vmin = vmin, vmax = vmax)
    im4 = ax[3].imshow(target_dict[im_num], cmap = 'gray', )
    ax[0].set_title('Orig')
    ax[1].set_title('Independent')
    ax[2].set_title('Dependent')
    ax[3].set_title('GT')
    fig.show()
    print(f'Min: I:{torch.min(tensor_dict1[im_num][0][0]).item():7.5f} D: {torch.min(tensor_dict2[im_num][0][0]).item():7.5f}', )
    print(f'Min: I:{torch.max(tensor_dict1[im_num][0][0]).item():7.5f} D: {torch.max(tensor_dict2[im_num][0][0]).item():7.5f}', )

In [5]:
def pseudo_anim(tensor_dict: dict, val_num: int = 0 , im_num: int = 0 ):
    ''' creates an animated gif using images
    
    output:
    creates a gif'''

    fig, ax = plt.subplots(1,1, figsize = (8, 8))
    ax.imshow(tensor_dict[val_num][im_num][0][0].numpy(), cmap = 'gray', vmin = 0, vmax = 1)
    fig.show()
    

Root Directory - change if different (make sure corresponding internal files are also changed)

In [6]:
root = '../results/base_model'

print(os.listdir(root))

['model_info', 'lightning_logs', 'test_statistics', 'dropblock_uncertainty_i', 'rotational_uncertainty', 'test_statistics_32_32', 'test_statistics_128_128', 'test_statistics_64_64', 'test_statistics_256_256', 'dropblock_uncertainty_d']


Validation Images

In [7]:
val_root_images = '../augmented_data/val/images'
orig = {}
# retrieves validation images as grayscale
for im in os.listdir(val_root_images):
    orig[int(im.split('_')[0])] = np.array(Image.open(join(val_root_images, im)).convert('L'))

val_root_targets = '../augmented_data/val/targets'
targets = {}
for im in os.listdir(val_root_targets):
    targets[int(im.split('_')[0])] = np.array(Image.open(join(val_root_targets, im)).convert('L'))


# Monte Carlo Dropblock - Dependent Channels

In [8]:
DB_d_path = join(root, 'dropblock_uncertainty_d/tensors')

DB_d_mean_tensors = extract_tensors(DB_d_path, 'mean.pt')
DB_d_tensors = extract_tensors(DB_d_path, 'std.pt')
DB_d_extra_tensors = extract_tensors(DB_d_path, 'tensors.pt')

Means

In [9]:
x = interact(display_tensor,
        tensor_dict=fixed(DB_d_mean_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(DB_d_mean_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01))

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

STD

In [10]:
x = interact(display_tensor,
        tensor_dict=fixed(DB_d_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(DB_d_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

View 25 Examples

In [11]:
x = interact(pseudo_anim, 
                tensor_dict = fixed(DB_d_extra_tensors),
                val_num =  (0,len(DB_d_extra_tensors) -1),
                im_num = (0, len(DB_d_extra_tensors[0]) -1))

interactive(children=(IntSlider(value=0, description='val_num', max=5), IntSlider(value=0, description='im_num…

# Monte Carlo Dropblock = Independent

In [12]:
DB_i_path = join(root, 'dropblock_uncertainty_i/tensors')

DB_i_mean_tensors = extract_tensors(DB_i_path, 'mean.pt')
DB_i_tensors = extract_tensors(DB_i_path, 'std.pt')
DB_i_extra_tensors = extract_tensors(DB_i_path, 'tensors.pt')

Mean

In [13]:
x = interact(display_tensor,
        tensor_dict=fixed(DB_i_mean_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(DB_i_mean_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

STD

In [14]:
x = interact(display_tensor,
        tensor_dict=fixed(DB_i_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(DB_i_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

View 25 Tensors

In [15]:
interact(pseudo_anim, 
                tensor_dict = fixed(DB_i_extra_tensors),
                val_num =  (0,len(DB_i_extra_tensors) -1),
                im_num = (0, len(DB_i_extra_tensors[0]) -1))

interactive(children=(IntSlider(value=0, description='val_num', max=5), IntSlider(value=0, description='im_num…

<function __main__.pseudo_anim(tensor_dict: dict, val_num: int = 0, im_num: int = 0)>

# Independent vs Dependent

Mean

In [16]:
x = interact(compare_tensor,
        tensor_dict1=fixed(DB_i_mean_tensors),
        tensor_dict2=fixed(DB_d_mean_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(DB_i_mean_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

STD

In [17]:
x = interact(compare_tensor,
        tensor_dict1=fixed(DB_i_tensors),
        tensor_dict2=fixed(DB_d_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(DB_i_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

# Rotational Uncertainty

In [18]:
rotation_path = join(root, 'rotational_uncertainty')

rotation_mean_tensors = extract_tensors(rotation_path, 'mean.pt')
rotation_tensors = extract_tensors(rotation_path, 'std.pt')
rotation_extra_tensors = extract_tensors(rotation_path, 'tensors.pt')

In [19]:
rotation_mean_tensors[0].shape

torch.Size([1, 1, 584, 565])

Mean

In [20]:
x = interact(display_tensor,
        tensor_dict=fixed(rotation_mean_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(rotation_mean_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

STD

In [21]:
x = interact(display_tensor,
        tensor_dict=fixed(rotation_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(rotation_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

View 25 Images

In [22]:
x = interact(pseudo_anim, 
                tensor_dict = fixed(rotation_extra_tensors),
                val_num =  (0,len(rotation_extra_tensors) -1),
                im_num = (0, len(rotation_extra_tensors[0]) -1))

interactive(children=(IntSlider(value=0, description='val_num', max=5), IntSlider(value=0, description='im_num…

# Testing Function

In [23]:

def visualize_magnitudes(
                tensor_dicts,
                im_dict,
                target_dict,
                im_num = 0,
                vmin = 0,
                vmax = 1):
        arr = len(tensor_dicts) + 2

        cols = 4
        rows = math.ceil(arr / 4)

        fig, axes = plt.subplots(rows, cols, figsize = (8 * cols, 8*rows))

        axes = axes.flatten()

        axes[0].imshow(im_dict[im_num], cmap = 'gray',)
        axes[0].set_title('Input Image')
        for model_id, (model_name, model_dict) in enumerate(tensor_dicts.items()):
                model_id = model_id + 1
                axes[model_id].imshow(model_dict[im_num][0][0].numpy(), cmap = 'gray', vmin = vmin, vmax = vmax)
                axes[model_id].set_title(model_name)
        axes[-1].imshow(target_dict[im_num], cmap = 'gray',)
        axes[-1].set_title('GT Image')
        
        fig.show()

In [24]:
def calculate_magnitudes(std_dicts, mean_dicts):

        cols = ['model_name', 'im_num', 'min', 'max','mean', 'std',  'mean_thresholded_0.01', 'std_thresholded_0.01','mean_thresholded_0.001','std_thresholded_0.001','mean_thresholded_0','std_thresholded_0',]
        data = dict(zip(cols, [[] for i in range(len(cols))]))
        data2 = dict(zip(cols, [[] for i in range(len(cols))]))

        for model_id, (model_name, model_dict) in enumerate(std_dicts.items()):
                for im_num in model_dict:
                        
                        std_data = model_dict[im_num][0][0].flatten()
                        data['model_name'].append(model_name)
                        data['im_num'].append(im_num)
                        data['min'].append(torch.min(std_data).item())
                        data['max'].append(torch.max(std_data).item())
                        data['mean'].append(torch.mean(std_data).item())
                        data['std'].append(torch.std(std_data).item())
                        data['mean_thresholded_0.01'].append(torch.mean(std_data[std_data > 0.01]).item())
                        data['std_thresholded_0.01'].append(torch.std(std_data[std_data > 0.01]).item())
                        data['mean_thresholded_0.001'].append(torch.mean(std_data[std_data > 0.001]).item())
                        data['std_thresholded_0.001'].append(torch.std(std_data[std_data > 0.001]).item())
                        data['mean_thresholded_0'].append(torch.mean(std_data[std_data > 0]).item())
                        data['std_thresholded_0'].append(torch.std(std_data[std_data > 0]).item())

        for model_id, (model_name, model_dict) in enumerate(mean_dicts.items()):
                for im_num in model_dict:
                        mean_data = model_dict[im_num][0][0].flatten()
                        data2['model_name'].append(model_name)
                        data2['im_num'].append(im_num)
                        data2['min'].append(torch.min(mean_data).item())
                        data2['max'].append(torch.max(mean_data).item())
                        data2['mean'].append(torch.mean(mean_data).item())
                        data2['std'].append(torch.std(mean_data).item())
                        data2['mean_thresholded_0.01'].append(torch.mean(mean_data[mean_data > 0.01]).item())
                        data2['std_thresholded_0.01'].append(torch.std(mean_data[mean_data > 0.01]).item())
                        data2['mean_thresholded_0.001'].append(torch.mean(mean_data[mean_data > 0.001]).item())
                        data2['std_thresholded_0.001'].append(torch.std(mean_data[mean_data > 0.001]).item())
                        data2['mean_thresholded_0'].append(torch.mean(mean_data[mean_data > 0]).item())
                        data2['std_thresholded_0'].append(torch.std(mean_data[mean_data > 0]).item())
        
        return pd.DataFrame(data, columns = cols), pd.DataFrame(data2, columns = cols)

In [25]:
def display_std_hist(std_dict, key = 'base_model', im_num = 0, threshold = .01):
    data = std_dict[key][im_num].flatten()
    data = data[data > threshold]
    print("Data size:", len(data))
    plt.hist(data)

In [26]:
def add_data(path, name, df):
    temp = pd.read_csv(path, index_col = 0)
    temp['name'] = name
    return pd.concat([df, temp])

# Test 1 - equivalent data, rf trained with no knowledge of higher fidelity

In [27]:
import pandas as pd

In [28]:
all_data = pd.DataFrame()
mean_db_tensors = {}
std_db_tensors = {}
mean_rot_tensors = {}
std_rot_tensors = {}


In [29]:
drive_root = 'DRIVE/'
#models
for model_id in 'BM-1 BM-2 BM-3 MF-1 MF-2 LF-1 LF-2 LF-3 LF-4 LF-5 LF-6'.split():
    path = join(drive_root, model_id)
    # reg
    all_data = add_data(join(path, 'statistics/val_images/metrics.csv'), name = model_id, df =  all_data)
    # db
    all_data = add_data(join(path, 'dropblock_uncertainty/statistics/val_images/metrics.csv'), name = 'base_model_db', df = all_data)
    mean_db_tensors[model_id] = extract_tensors(join(path, 'dropblock_uncertainty/tensors'), 'mean.pt')
    std_db_tensors[model_id] = extract_tensors(join(path, 'dropblock_uncertainty/tensors'), 'std.pt')
    mean_rot_tensors[model_id] = extract_tensors(join(path, 'rotational_uncertainty'), 'mean.pt')
    std_rot_tensors[model_id] = extract_tensors(join(path, 'rotational_uncertainty'), 'std.pt')

In [32]:
all_data.groupby(['name']).mean()

,F1_Vessel,AUROC_Vessel,Accuracy_Vessel
name,,,
128_128,0.524038,0.733808,0.922568
128_128_db,0.459324,0.675947,0.925157
256_256,0.710415,0.818374,0.954858
256_256_db,0.679832,0.778285,0.954762
32_32,0.093510,0.520609,0.903494
32_32_db,0.078163,0.516654,0.905484
64_64,0.325207,0.615942,0.907267
64_64_db,0.275221,0.589005,0.910146
base_model,0.806247,0.889031,0.967834


In [33]:
x = interact_manual(visualize_magnitudes,
        tensor_dicts=fixed(mean_db_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(mean_db_tensors) - 1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

In [34]:
x = interact_manual(visualize_magnitudes,
        tensor_dicts=fixed(std_db_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(std_db_tensors) - 1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

In [35]:
x = interact_manual(visualize_magnitudes,
        tensor_dicts=fixed(mean_rot_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(mean_rot_tensors) - 1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

In [36]:
x = interact_manual(visualize_magnitudes,
        tensor_dicts=fixed(std_rot_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(std_rot_tensors) - 1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

In [37]:
x = interact(display_std_hist, 
            std_dict = fixed(std_db_tensors),
            key = list(std_db_tensors.keys()),
            im_num = (0, len(std_db_tensors['base_model']) - 1),
            threshold = (0, 1, .001))


interactive(children=(Dropdown(description='key', options=('base_model', 'rf_model_32_32', 'rf_model_64_64', '…

In [38]:
db_std_mag, db_mean_mag = calculate_magnitudes(std_db_tensors, mean_db_tensors)
rot_std_mag, rot_mean_mag = calculate_magnitudes(std_rot_tensors, mean_rot_tensors)

In [39]:
db_mean_mag.groupby('model_name').mean()

,im_num,min,max,mean,std,mean_thresholded_0.01,std_thresholded_0.01,mean_thresholded_0.001,std_thresholded_0.001,mean_thresholded_0,std_thresholded_0
model_name,,,,,,,,,,,
base_model,2.5,0.0,0.999766,0.081301,0.228359,0.335703,0.364631,0.146602,0.290586,0.118181,0.267278
mf_model,2.5,0.0,0.999835,0.075817,0.220392,0.273149,0.351702,0.132724,0.278341,0.110210,0.258475
rf_model_128_128,2.5,0.0,0.999077,0.086054,0.177588,0.166840,0.220321,0.126442,0.202905,0.125088,0.202291
rf_model_256_256,2.5,0.0,0.999302,0.077329,0.191877,0.179030,0.263083,0.115858,0.225009,0.112406,0.222547
rf_model_32_32,2.5,0.0,0.952298,0.080087,0.115916,0.116679,0.123724,0.116420,0.123689,0.116409,0.123688
rf_model_64_64,2.5,0.0,0.990090,0.082775,0.153431,0.134050,0.177408,0.120393,0.172343,0.120318,0.172313


In [40]:
db_std_mag.groupby('model_name').mean()

,im_num,min,max,mean,std,mean_thresholded_0.01,std_thresholded_0.01,mean_thresholded_0.001,std_thresholded_0.001,mean_thresholded_0,std_thresholded_0
model_name,,,,,,,,,,,
base_model,2.5,0.0,0.314011,0.012924,0.026978,0.049251,0.036683,0.021783,0.032271,0.018783,0.030784
mf_model,2.5,0.0,0.306536,0.018456,0.037292,0.058651,0.049695,0.030485,0.043989,0.026826,0.042380
rf_model_128_128,2.5,0.0,0.286140,0.030790,0.045105,0.062749,0.048406,0.045340,0.048307,0.044759,0.048272
rf_model_256_256,2.5,0.0,0.334632,0.025472,0.040994,0.057627,0.047330,0.037985,0.045009,0.037027,0.044829
rf_model_32_32,2.5,0.0,0.246839,0.027603,0.033890,0.043499,0.034191,0.040123,0.034163,0.040122,0.034164
rf_model_64_64,2.5,0.0,0.297770,0.029466,0.041686,0.055781,0.044646,0.042847,0.044187,0.042832,0.044186


In [41]:
rot_mean_mag.groupby('model_name').mean()

,im_num,min,max,mean,std,mean_thresholded_0.01,std_thresholded_0.01,mean_thresholded_0.001,std_thresholded_0.001,mean_thresholded_0,std_thresholded_0
model_name,,,,,,,,,,,
base_model,2.5,0.0,0.999994,0.088071,0.236270,0.358149,0.364237,0.184553,0.314785,0.128018,0.275717
mf_model,2.5,0.0,0.999989,0.090559,0.239932,0.315408,0.363640,0.151134,0.294884,0.131635,0.279761
rf_model_128_128,2.5,0.0,0.998772,0.093570,0.188161,0.167324,0.228217,0.136529,0.213783,0.136015,0.213557
rf_model_256_256,2.5,0.0,0.999863,0.086979,0.214741,0.209426,0.296441,0.132890,0.253748,0.126442,0.248975
rf_model_32_32,2.5,0.0,0.774203,0.067328,0.075885,0.097863,0.073352,0.097863,0.073352,0.097863,0.073352
rf_model_64_64,2.5,0.0,0.978089,0.073851,0.127798,0.108935,0.142351,0.107347,0.141813,0.107347,0.141813


In [42]:
rot_std_mag.groupby('model_name').mean()

,im_num,min,max,mean,std,mean_thresholded_0.01,std_thresholded_0.01,mean_thresholded_0.001,std_thresholded_0.001,mean_thresholded_0,std_thresholded_0
model_name,,,,,,,,,,,
base_model,2.5,0.0,0.274130,0.006299,0.014207,0.030917,0.019141,0.017481,0.019355,0.009155,0.016343
mf_model,2.5,0.0,0.227478,0.008704,0.017743,0.037510,0.021791,0.018646,0.022380,0.012650,0.020184
rf_model_128_128,2.5,0.0,0.291147,0.028735,0.040414,0.056920,0.042629,0.042707,0.042769,0.041773,0.042699
rf_model_256_256,2.5,0.0,0.288272,0.015916,0.025269,0.041827,0.028435,0.025753,0.028115,0.023137,0.027521
rf_model_32_32,2.5,0.0,0.242445,0.020675,0.030082,0.038507,0.033995,0.030051,0.032131,0.030051,0.032131
rf_model_64_64,2.5,0.0,0.331070,0.026159,0.040596,0.052759,0.046494,0.038026,0.044056,0.038022,0.044055


# Test 2

In [43]:
all_data = pd.DataFrame()
mean_db_tensors = {}
std_db_tensors = {}
mean_rot_tensors = {}
std_rot_tensors = {}

In [44]:
#base model
for folder in ['_.25', '_1', '_.0625']:
    db_i_path = f'../results/Test2/base_model{folder}/'

    # reg
    all_data = add_data(join(db_i_path, 'statistics/val_images/metrics.csv'), name = f'base_model{folder}', df =  all_data)

    # db
    all_data = add_data(join(db_i_path, 'dropblock_uncertainty/statistics/val_images/metrics.csv'), name = f'base_model{folder}_db', df = all_data)
    mean_db_tensors[f'base_model{folder}'] = extract_tensors(join(db_i_path, 'dropblock_uncertainty/tensors'), 'mean.pt')
    std_db_tensors[f'base_model{folder}'] = extract_tensors(join(db_i_path, 'dropblock_uncertainty/tensors'), 'std.pt')
    mean_rot_tensors[f'base_model{folder}'] = extract_tensors(join(db_i_path, 'rotational_uncertainty'), 'mean.pt')
    std_rot_tensors[f'base_model{folder}'] = extract_tensors(join(db_i_path, 'rotational_uncertainty'), 'std.pt')

In [45]:
for folder in ['128_128_1','256_256_.25','256_256_1']:
    # 32x32, ...
    path = f'../results/Test2/rf_model_{folder}'

    # reg 
    all_data = add_data(join(path, 'statistics/statistics_normal/val_images/metrics.csv'), name = f'rf_model_{folder}', df =  all_data)

    # temp
    all_data = add_data(join(path, 'dropblock_uncertainty/statistics/val_images/metrics.csv'),name = f'rf_model_{folder}_db', df =  all_data)
    mean_db_tensors[f'rf_model_{folder}'] = extract_tensors(join(path, 'dropblock_uncertainty/tensors'), 'mean.pt')
    std_db_tensors[f'rf_model_{folder}'] = extract_tensors(join(path, 'dropblock_uncertainty/tensors'), 'std.pt')
    mean_rot_tensors[f'rf_model_{folder}'] = extract_tensors(join(path, 'rotational_uncertainty'), 'mean.pt')
    std_rot_tensors[f'rf_model_{folder}'] = extract_tensors(join(path, 'rotational_uncertainty'), 'std.pt')

In [46]:
#mf model
mf_path = '../results/Test2/mf_model/'

# reg 
all_data = add_data(join(mf_path, 'statistics/statistics_normal/val_images/metrics.csv'), name = 'mf_model', df =  all_data)

# temp
all_data = add_data(join(mf_path, 'dropblock_uncertainty/statistics/val_images/metrics.csv'),name = f'mf_model_db', df =  all_data)
mean_db_tensors['mf_model'] = extract_tensors(join(mf_path, 'dropblock_uncertainty/tensors'), 'mean.pt')
std_db_tensors['mf_model'] = extract_tensors(join(mf_path, 'dropblock_uncertainty/tensors'), 'std.pt')
mean_rot_tensors['mf_model'] = extract_tensors(join(mf_path, 'rotational_uncertainty'), 'mean.pt')
std_rot_tensors['mf_model'] = extract_tensors(join(mf_path, 'rotational_uncertainty'), 'std.pt')

In [47]:
all_data.groupby(['name']).mean()

,F1_Vessel,AUROC_Vessel,Accuracy_Vessel
name,,,
base_model_.0625,0.777135,0.865662,0.964051
base_model_.0625_db,0.718423,0.799434,0.959787
base_model_.25,0.803741,0.893278,0.967089
base_model_.25_db,0.795954,0.867031,0.967621
base_model_1,0.803757,0.884405,0.967700
base_model_1_db,0.797923,0.868913,0.967789
mf_model,0.802595,0.879663,0.967887
mf_model_db,0.794788,0.863990,0.967612
rf_model_128_128_1,0.518001,0.729046,0.922324


In [48]:
x = interact_manual(visualize_magnitudes,
        tensor_dicts=fixed(mean_db_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(mean_db_tensors) - 1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=6), FloatSlider(value=0.0, description='vmi…

In [49]:
x = interact_manual(visualize_magnitudes,
        tensor_dicts=fixed(std_db_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(std_db_tensors) - 1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=6), FloatSlider(value=0.0, description='vmi…

In [50]:
x = interact_manual(visualize_magnitudes,
        tensor_dicts=fixed(mean_rot_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(mean_rot_tensors) - 1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=6), FloatSlider(value=0.0, description='vmi…

In [51]:
x = interact_manual(visualize_magnitudes,
        tensor_dicts=fixed(std_rot_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(std_rot_tensors) - 1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=6), FloatSlider(value=0.0, description='vmi…

In [52]:
x = interact(display_std_hist, 
            std_dict = fixed(std_db_tensors),
            key = list(std_db_tensors.keys()),
            im_num = (0, len(std_db_tensors['base_model_1']) - 1),
            threshold = (0, 1, .001))


interactive(children=(Dropdown(description='key', options=('base_model_.25', 'base_model_1', 'base_model_.0625…

In [53]:
db_std_mag, db_mean_mag = calculate_magnitudes(std_db_tensors, mean_db_tensors)
rot_std_mag, rot_mean_mag = calculate_magnitudes(std_rot_tensors, mean_rot_tensors)

In [54]:
db_mean_mag.groupby('model_name').mean()

,im_num,min,max,mean,std,mean_thresholded_0.01,std_thresholded_0.01,mean_thresholded_0.001,std_thresholded_0.001,mean_thresholded_0,std_thresholded_0
model_name,,,,,,,,,,,
base_model_.0625,2.5,0.0,0.986358,0.073270,0.186516,0.143478,0.243521,0.106534,0.216880,0.106502,0.216856
base_model_.25,2.5,0.0,0.998973,0.081503,0.222536,0.273929,0.341553,0.123577,0.264385,0.118475,0.260006
base_model_1,2.5,0.0,0.999399,0.081400,0.222742,0.283823,0.343921,0.130523,0.270492,0.118325,0.260279
mf_model,2.5,0.0,0.999529,0.082953,0.222265,0.249057,0.330461,0.122327,0.260806,0.120581,0.259362
rf_model_128_128_1,2.5,0.0,0.999937,0.088003,0.185005,0.182728,0.233359,0.136030,0.214924,0.127919,0.211217
rf_model_256_256_.25,2.5,0.0,0.999913,0.084473,0.207979,0.219549,0.290766,0.142673,0.254105,0.122792,0.241135
rf_model_256_256_1,2.5,0.0,0.999962,0.078294,0.201123,0.203279,0.285547,0.130495,0.245813,0.113811,0.233941


In [55]:
db_std_mag.groupby('model_name').mean()

,im_num,min,max,mean,std,mean_thresholded_0.01,std_thresholded_0.01,mean_thresholded_0.001,std_thresholded_0.001,mean_thresholded_0,std_thresholded_0
model_name,,,,,,,,,,,
base_model_.0625,2.5,0.0,0.290920,0.029831,0.045042,0.053042,0.050470,0.043360,0.048596,0.043360,0.048596
base_model_.25,2.5,0.0,0.297801,0.016873,0.031429,0.050680,0.040155,0.024989,0.035505,0.024525,0.035324
base_model_1,2.5,0.0,0.278446,0.014686,0.027340,0.046517,0.034737,0.022515,0.031183,0.021345,0.030727
mf_model,2.5,0.0,0.286063,0.019093,0.033174,0.051423,0.040247,0.027941,0.036906,0.027751,0.036852
rf_model_128_128_1,2.5,0.0,0.319807,0.035429,0.052351,0.071200,0.056033,0.053616,0.056270,0.051501,0.056164
rf_model_256_256_.25,2.5,0.0,0.343904,0.033238,0.057707,0.080075,0.067859,0.053731,0.065382,0.048316,0.064107
rf_model_256_256_1,2.5,0.0,0.357455,0.027951,0.049488,0.069391,0.059454,0.045081,0.056310,0.040632,0.055153


In [56]:
rot_mean_mag.groupby('model_name').mean()

,im_num,min,max,mean,std,mean_thresholded_0.01,std_thresholded_0.01,mean_thresholded_0.001,std_thresholded_0.001,mean_thresholded_0,std_thresholded_0
model_name,,,,,,,,,,,
base_model_.0625,2.5,0.0,0.998944,0.089509,0.225490,0.232939,0.317766,0.136383,0.266632,0.130107,0.261963
base_model_.25,2.5,0.0,0.999907,0.092224,0.238747,0.337280,0.358198,0.158430,0.295785,0.134055,0.277928
base_model_1,2.5,0.0,0.999965,0.087962,0.232245,0.325827,0.352977,0.164448,0.297105,0.127861,0.270728
mf_model,2.5,0.0,0.999852,0.087938,0.230232,0.306359,0.346928,0.131557,0.271182,0.127826,0.268230
rf_model_128_128_1,2.5,0.0,0.999203,0.097572,0.188901,0.160544,0.221375,0.142211,0.213495,0.141832,0.213360
rf_model_256_256_.25,2.5,0.0,0.999993,0.094628,0.225453,0.238703,0.310335,0.154623,0.271578,0.137553,0.260625
rf_model_256_256_1,2.5,0.0,0.999992,0.090186,0.220066,0.208271,0.298594,0.140191,0.261240,0.131102,0.254948


In [57]:
rot_std_mag.groupby('model_name').mean()

,im_num,min,max,mean,std,mean_thresholded_0.01,std_thresholded_0.01,mean_thresholded_0.001,std_thresholded_0.001,mean_thresholded_0,std_thresholded_0
model_name,,,,,,,,,,,
base_model_.0625,2.5,0.0,0.238858,0.011278,0.022376,0.041025,0.029488,0.020017,0.026852,0.016392,0.025373
base_model_.25,2.5,0.0,0.276573,0.007111,0.015520,0.032566,0.020723,0.017360,0.020471,0.010334,0.017795
base_model_1,2.5,0.0,0.252663,0.006770,0.014501,0.030590,0.019115,0.016451,0.018982,0.009839,0.016595
mf_model,2.5,0.0,0.198608,0.007641,0.014732,0.031201,0.017297,0.016517,0.018248,0.011105,0.016636
rf_model_128_128_1,2.5,0.0,0.328926,0.033673,0.043456,0.059936,0.043899,0.049252,0.044562,0.048951,0.044597
rf_model_256_256_.25,2.5,0.0,0.294888,0.021769,0.036454,0.056282,0.041737,0.036943,0.041366,0.031642,0.040197
rf_model_256_256_1,2.5,0.0,0.324648,0.020102,0.032311,0.049843,0.036508,0.032427,0.035958,0.029221,0.035326
